In [1]:
import geopandas as gpd

In [2]:
!wget https://gaftp.epa.gov/EPADataCommons/ORD/Ecoregions/us/us_eco_l3.zip

--2024-03-13 06:37:54--  https://gaftp.epa.gov/EPADataCommons/ORD/Ecoregions/us/us_eco_l3.zip
Resolving gaftp.epa.gov (gaftp.epa.gov)... 134.67.100.99, 2620:117:506f:c7::f063
Connecting to gaftp.epa.gov (gaftp.epa.gov)|134.67.100.99|:443... connected.
ERROR: cannot verify gaftp.epa.gov's certificate, issued by ‘CN=DigiCert Global G2 TLS RSA SHA256 2020 CA1,O=DigiCert Inc,C=US’:
  Unable to locally verify the issuer's authority.
To connect to gaftp.epa.gov insecurely, use `--no-check-certificate'.


In [3]:
eco_regions = gpd.read_file("zip://" + "../us_eco_l3.zip")
southern_rockies = eco_regions[eco_regions['US_L3NAME'] == "Southern Rockies"]
southern_rockies = southern_rockies.to_crs("EPSG:4326")
southern_rockies.to_file("/data-store/output/southern_rockies.geojson", driver="GeoJSON")

# intersect with GLAD ARD Tiles
ard_tiles = gpd.read_file("https://glad.umd.edu/users/Potapov/ARD/Global_ARD_tiles.zip")
relevant_ard_tiles = ard_tiles.clip(southern_rockies)

In [4]:
relevant_ard_tiles.explore()

In [5]:
import pandas as pd

In [7]:
import numpy as np
tiles = relevant_ard_tiles['TILE'].unique()

start_year = 1997
end_year = 2023
start_codes = np.cumsum(np.repeat(23, end_year-start_year + 1)) + 392 - 23
year_code_maps = {year: list(range(start, start + 23)) for year, start in  zip(range(start_year, end_year+1), start_codes)}

pattern = "https://glad.umd.edu/dataset/glad_ard2/{lat}/{tile}/{period}.tif"

rows = []
for tile in tiles:
    for year, year_codes in year_code_maps.items():
        for code in year_codes:
            rows.append((tile, year, code, pattern.format(lat=tile.split("_")[1], tile=tile, period=code)))

ard_assets = pd.DataFrame(rows, columns=['tile','year','16-day-code','url'])

In [15]:
ard_assets.iloc[:100]['url'].to_csv("/data", index=False)